In [2]:
import pandas as pd
import numpy as np

In [3]:
# read data
schools = pd.read_csv('./raw_data/schools_complete.csv')
students = pd.read_csv('./raw_data/students_complete.csv')

In [4]:
# District-level school-level data
num_schools = schools['School ID'].count()
total_budget = schools['budget'].sum()

In [5]:
# Merge data (student-level)
school_students = pd.merge(schools, students, left_on='name', right_on='school', how='left')
school_students.rename(columns={'name_x': 'School Name', 'name_y': 'Student Name'}, inplace=True)

# Add passing columns, assuming >= 70 is passing and one must pass both reading and math to pass overall
school_students['Passing Reading'] = school_students['reading_score'] >= 70
school_students['Passing Math'] = school_students['math_score'] >= 70
school_students['Passing Overall'] = school_students['Passing Math'] & school_students['Passing Reading']

# Get district-level passing counts
pass_math = school_students[school_students['Passing Math']]['Student ID'].count()
pass_reading = school_students[school_students['Passing Reading']]['Student ID'].count()
pass_total = school_students[school_students['Passing Overall']]['Student ID'].count()

In [8]:
# Total-level group includes count of students and average math and reading scores.
funcs = {'Student ID': 'count', 'math_score': 'mean', 'reading_score': 'mean'}

# Hacky way to group without a groupby column (aggregate to district level)
ss_total_group = school_students.groupby(by=lambda x: 0)

# aggregate to count students and get means of math and reading scores
ss_total = ss_total_group.agg(funcs)
ss_total.rename(columns={'reading_score': 'Average Reading Score', 'math_score': 'Average Math Score'
                        ,'Student ID': 'Total Students'}, inplace=True)

# Merge student- and school-level data
ss_total['% Passing Math'] = pass_math / ss_total['Total Students']
ss_total['% Passing Reading'] = pass_reading / ss_total['Total Students']
ss_total['% Passing Overall'] = pass_total / ss_total['Total Students']
ss_total['Total Budget'] = total_budget
ss_total['Total Schools'] = num_schools
ss_total['% Passing Math'] = ss_total['% Passing Math'].map('{:,.2%}'.format)
# map('${:,.2f}'.format)
# ss_total['Average Reading Score'] = ss_total['Average Reading Score'].map('${:,.2%}'.format)
ss_total[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Passing Overall']]

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.98%,0.858055,0.651723


In [23]:
# school-level data
school_result = schools[['name', 'type', 'budget']]

# student-level data by school
school_students_gb = school_students.groupby(by='School ID')
funcs = {'Student ID': 'count', 'math_score': 'mean', 'reading_score': 'mean', 'Passing Math': 'sum', 'Passing Reading': 'sum', 'Passing Overall': 'sum'}#, 'Passing Reading': 'count', 'pass_overall': 'count'}

school_totals = school_students_gb.agg(funcs)

school_result = pd.merge(schools[['name', 'type', 'budget']], school_totals, left_index=True, right_index=True)
school_result['Per Student Budget'] = school_result['budget'] / school_result['Student ID']
school_result['% Passing Math'] = school_result['Passing Math'] / school_result['Student ID']
school_result['% Passing Reading'] = school_result['Passing Reading'] / school_result['Student ID']
school_result['% Passing Overall'] = school_result['Passing Overall'] / school_result['Student ID']
# school_totals
school_result

,name,type,budget,math_score,Passing Reading,Passing Overall,reading_score,Passing Math,Student ID,Per Student Budget,% Passing Math,% Passing Reading,% Passing Overall
0,Huang High School,District,1910635,76.629414,2372.0,1561.0,81.182722,1916.0,2917,655.0,0.656839,0.813164,0.535139
1,Figueroa High School,District,1884411,76.711767,2381.0,1569.0,81.158020,1946.0,2949,639.0,0.659885,0.807392,0.532045
2,Shelton High School,Charter,1056600,83.359455,1688.0,1583.0,83.725724,1653.0,1761,600.0,0.938671,0.958546,0.898921
3,Hernandez High School,District,3022020,77.289752,3748.0,2481.0,80.934412,3094.0,4635,652.0,0.667530,0.808630,0.535275
4,Griffin High School,Charter,917500,83.351499,1426.0,1330.0,83.816757,1371.0,1468,625.0,0.933924,0.971390,0.905995
5,Wilson High School,Charter,1319574,83.274201,2204.0,2068.0,83.989488,2143.0,2283,578.0,0.938677,0.965396,0.905826
6,Cabrera High School,Charter,1081356,83.061895,1803.0,1697.0,83.975780,1749.0,1858,582.0,0.941335,0.970398,0.913348
7,Bailey High School,District,3124928,77.048432,4077.0,2719.0,81.033963,3318.0,4976,628.0,0.666801,0.819333,0.546423
8,Holden High School,Charter,248087,83.803279,411.0,381.0,83.814988,395.0,427,581.0,0.925059,0.962529,0.892272
9,Pena High School,Charter,585858,83.839917,923.0,871.0,84.044699,910.0,962,609.0,0.945946,0.959459,0.905405


In [30]:
top_performing = school_result.sort_values(by='% Passing Overall', ascending=False).head()
top_performing

,name,type,budget,math_score,Passing Reading,Passing Overall,reading_score,Passing Math,Student ID,Per Student Budget,% Passing Math,% Passing Reading,% Passing Overall
6,Cabrera High School,Charter,1081356,83.061895,1803.0,1697.0,83.975780,1749.0,1858,582.0,0.941335,0.970398,0.913348
14,Thomas High School,Charter,1043130,83.418349,1591.0,1487.0,83.848930,1525.0,1635,638.0,0.932722,0.973089,0.909480
4,Griffin High School,Charter,917500,83.351499,1426.0,1330.0,83.816757,1371.0,1468,625.0,0.933924,0.971390,0.905995
5,Wilson High School,Charter,1319574,83.274201,2204.0,2068.0,83.989488,2143.0,2283,578.0,0.938677,0.965396,0.905826
9,Pena High School,Charter,585858,83.839917,923.0,871.0,84.044699,910.0,962,609.0,0.945946,0.959459,0.905405


In [37]:
bottom_performing = school_result.sort_values(by='% Passing Overall').head()
bottom_performing

,name,type,budget,math_score,Passing Reading,Passing Overall,reading_score,Passing Math,Student ID,Per Student Budget,% Passing Math,% Passing Reading,% Passing Overall
11,Rodriguez High School,District,2547363,76.842711,3208.0,2119.0,80.744686,2654.0,3999,637.0,0.663666,0.802201,0.529882
1,Figueroa High School,District,1884411,76.711767,2381.0,1569.0,81.158020,1946.0,2949,639.0,0.659885,0.807392,0.532045
0,Huang High School,District,1910635,76.629414,2372.0,1561.0,81.182722,1916.0,2917,655.0,0.656839,0.813164,0.535139
3,Hernandez High School,District,3022020,77.289752,3748.0,2481.0,80.934412,3094.0,4635,652.0,0.667530,0.808630,0.535275
12,Johnson High School,District,3094650,77.072464,3867.0,2549.0,80.966394,3145.0,4761,650.0,0.660576,0.812224,0.535392


In [38]:
school_students

,School ID,School Name,type,size,budget,Student ID,Student Name,gender,grade,school,reading_score,math_score,Passing Reading,Passing Math,Passing Overall
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,Huang High School,66,79,False,True,False
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,Huang High School,94,61,True,False,False
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,Huang High School,90,60,True,False,False
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False,False
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True,True
5,0,Huang High School,District,2917,1910635,5,Bryan Miranda,M,9th,Huang High School,94,94,True,True,True
6,0,Huang High School,District,2917,1910635,6,Sheena Carter,F,11th,Huang High School,82,80,True,True,True
7,0,Huang High School,District,2917,1910635,7,Nicole Baker,F,12th,Huang High School,96,69,True,False,False
8,0,Huang High School,District,2917,1910635,8,Michael Roth,M,10th,Huang High School,95,87,True,True,True
9,0,Huang High School,District,2917,1910635,9,Matthew Greene,M,10th,Huang High School,96,84,True,True,True


In [43]:
pd.pivot_table(school_students, index='School Name', columns='grade', values='Passing Math')

grade,10th,11th,12th,9th
School Name,,,,
Bailey High School,0.663438,0.684253,0.642996,0.671468
Cabrera High School,0.939914,0.923237,0.950262,0.952652
Figueroa High School,0.665793,0.653032,0.685990,0.641355
Ford High School,0.693503,0.687405,0.654917,0.689076
Griffin High School,0.940887,0.941828,0.928082,0.924205
Hernandez High School,0.667482,0.668199,0.667377,0.667149
Holden High School,0.929825,0.912621,0.951807,0.913386
Huang High School,0.634941,0.647712,0.661538,0.681280
Johnson High School,0.651182,0.669449,0.650641,0.667857


In [45]:
pd.pivot_table(school_students, index='School Name', columns='grade', values='Passing Reading')

grade,10th,11th,12th,9th
School Name,,,,
Bailey High School,0.835351,0.805755,0.813230,0.821674
Cabrera High School,0.974249,0.970954,0.968586,0.967803
Figueroa High School,0.812582,0.781382,0.819646,0.815421
Ford High School,0.790960,0.799697,0.782931,0.795918
Griffin High School,0.972906,0.975069,0.979452,0.960880
Hernandez High School,0.806846,0.817096,0.797441,0.811143
Holden High School,0.964912,0.961165,0.987952,0.944882
Huang High School,0.821382,0.805825,0.811966,0.812796
Johnson High School,0.814996,0.796327,0.819444,0.818571


In [55]:
bins = [0, 1000, 2000, 5000]
names = ['Small (< 1000)', 'Medium (1000 - 2000)', 'Large (2000 - 5000)']

size_series = pd.cut(school_result['Student ID'], bins, labels=names)
school_result['School Size'] = size_series
size_groups = school_result.groupby(by='School Size')
size_groups['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Passing Overall'].mean()

,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (< 1000),83.821598,83.929843,0.935502,0.960994,0.898839
Medium (1000 - 2000),83.374684,83.864438,0.935997,0.967907,0.906215
Large (2000 - 5000),77.746417,81.344493,0.699634,0.827666,0.582860
